# A sample example of using sentence transformers to perform semantic search

In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
import faiss
import numpy as np

In [2]:
# Corpus with example sentences (description of top startups in Canada taken from https://www.foundersbeta.com/tech-companies/20-tech-companies-to-watch-for-in-2021/)
corpus = ['Hopintech. This tech company is tackling the commute. Hop In provides logistics software services to offer customized corporate shuttle solutions for the daily commute.',
          'LayerCI. (Y-Combinator backed) helps technology forward businesses disrupt their industries. They created the world’s first Continuous Staging platform – an easy-to-use and scalable cloud-based SaaS that provides every software developer with the best CI/CD + staging experience.',
          'SHIFT. SHIFT is a digital coach for remote teams. The suite of digital tools empowers leaders to create alignment and connection, evolve their culture, and build highly effective and outperforming virtual teams.',
          'Honeybee. Honeybee is shaping the future in research studies. You can take charge in shaping a happier and healthier community by taking part in research studies.',
          'Beeznests. The first student exclusive freelance platform. You can outsource all your remote tasks /projects to students here',
          'Hypeercare. Hypeercare is real-time care coordination for medical professionals, keeping providers in-sync to deliver the best care for patients.',
          'Lumaki Labs. Virtual internships are here. This EdTech startup is on a mission to revolutionize the future of work through helping companies unlock student potential, anywhere. Lumaki Labs offers a 2-in-1 virtual onboarding and recruitment platform to help organizations deliver mutually beneficial virtual internships.',
          'ODAIA. ODAIA is a pioneer in combining process mining, customer journey mapping and AI to deliver sales and marketing insights and process automation tools to enterprise companies globally. ODAIA is headquartered in Toronto, Canada and was founded out of the University of Toronto. ODAIA is led by an experienced group of serial entrepreneurs, data scientists and AI engineers and is trusted by leading global organizations. ',
          'Ruckify. Ruckify is the world’s largest peer-to-peer rental marketplace that aims to reduce unnecessary consumerism and provide people the ability to earn some extra cash from everyday items that they already own. At Ruckify, they believe in communities sharing, collaborating, and helping one another while promoting sustainability. You can rent items from fellow community members, or you can start your own side-hustle and post your own items for rent.',
          'AIRenty. AIRenty is a SaaS PropTech platform which streamline the Real Estate rental process with the help of AI. By automating and simplifying the workflow we make the search activity more enjoyable and less time consuming for everyone!',
          'MeetFox. MeetFox offers individual service professionals a simple, one-stop solution to digitize their business. With the vision of becoming the Shopify for professional services, MeetFox combines online scheduling, seamless video calls, and instant payments in one product that can be plugged in on any website and allowing currently 9000 professionals to deliver their services online without the need for technical skills.',
          'Mary Agrotechnologies. Mary Agrotechnologies is a Toronto-based vertical farming company with proprietary designs and data-driven automation that drastically increase energy efficiency and crop yields. Its automated home grow box is a leader in the vertical, and the company is the first North American company to receive a full set of strictly issued hemp licenses in China, from cultivation to extraction to sale & export.',
          'Getchat.app. Engagement with your website visitor and turning them into customers is key. This startup helps you to chat with your visitors on your site using WhatsApp. Getchat.app got the product of the day with Product Hunt.',
          'Unwrapit. What is the future of gifting? This startup has the solution. Unwrapit is a Toronto-based startup revealing the future of virtual corporate and event gifting',
          'Equivesto. Funding is big challenge for most startups but Equivesto is here to change that. Equivesto is an equity investment portal that connects early-stage companies and small businesses to the capital they need to grow through the crowd’s contributions. With Equivesto, Canadian companies of all industries can raise between $25,000 and 1.5MM, and turn their communities into investors with a minimum investment of $100 CAD.',
          'Local Logic. Local Logic is a location intelligence platform that quantifies the “sense of place” for any given location. With more than 20 billion unique data points — the largest unique location data set in the U.S. and Canada — Local Logic offers predictive analytics to inform real estate decision making in the built environment. The founding team includes urban planners and data scientists. ',
          'PointClickCare. From start-up to market leader, PointClickCare’s innovative cloud-based technology is advancing the senior care industry, and making a true impact on people’s lives. Recently named one of Deloitte’s fastest growing technology companies and Canada’s best managed companies, PointClickCare provides so many opportunities and a great culture for employees.',
          'NorthOne. NorthOne is a mobile-first, tech-powered bank account built for startups, freelancers, and small/medium-sized businesses that help them bank, manage their finances, and integrate all of their financial tools in a simple and intuitive way. Poor financial literacy has an outsized impact on the costs and failure rates amongst SMBs, and NorthOne is on a mission to eliminate these problems and allow business owners to focus on what really matters – growing a successful business. ',
          'Viafoura partners with over 600 media brands to engage, convert and monetize their digital audiences. With best-in-class engagement and content moderation solutions — including real-time conversations, live blogs, community chat, personalization tools and AI-powered moderation — Viafoura helps companies create active, civil and loyal online communities.',
          'Remitr is a Toronto-based fintech startup for business payments that offers a better alternative to wire transfers, cheque payments and bank visits. The Remitr Global Network allows businesses to make fast—often one day—payments across Canada and to over 150 countries. Co-founded by Kanchan Kumar and Sandeep Todi in 2016, Remitr now processes hundreds of millions of dollars annually and is backed by global investors.',
          ]

# Query sentences:
queries = ['online scheduling and manpower management', 'health and safety', 'outsourcing solutions']

In [3]:
# set the model of choise and create embeddings for the corpus
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
corpus_embeddings = model.encode(corpus)

In [4]:
#showing how the embeddings look like
print(corpus_embeddings[0][:50])

[ 0.07325883 -0.08525414 -0.09272305 -0.22334716 -0.14390835 -0.05396098
  0.16733107 -0.20756385 -0.10939638  0.04604813  0.02828181  0.24929447
  0.08974447 -0.48233297  0.20653023 -0.02335097  0.2533164   0.0892872
  0.27970624 -0.24050578 -0.12152178 -0.29349998 -0.17054443  0.12383818
 -0.00861715  0.05410332  0.02687951  0.24506442 -0.17733878 -0.43095306
 -0.02365507  0.37127167 -0.03754214  0.21769638  0.27611348  0.11021805
  0.05535251  0.03278187 -0.10101817 -0.3072224  -0.12462296 -0.2035871
 -0.03118398 -0.23672603  0.00080161 -0.08938362 -0.09032591 -0.02147989
 -0.13387482  0.27904788]


In [5]:
#embeddings of 384 dims 
corpus_embeddings.shape

(20, 384)

## Perform search using the sentence_transformers util.semantic_search

In [6]:
# Find the closest 5 sentences of the corpus for each query sentence 
# use util.semantic_search to perform cosine similarty + topk
top_k = 5
for query in queries:
    #create embeddings for each query 
    query_embedding = model.encode(query)
    # seach in the corpus
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k = top_k)[0]
    print("\n********************")
    print("Query:", query)
    print("\nTop 5 most similar entries in corpus:")
    
    for hit in hits:
        print("  - ", corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))


********************
Query: online scheduling and manpower management

Top 5 most similar entries in corpus:
  -  MeetFox. MeetFox offers individual service professionals a simple, one-stop solution to digitize their business. With the vision of becoming the Shopify for professional services, MeetFox combines online scheduling, seamless video calls, and instant payments in one product that can be plugged in on any website and allowing currently 9000 professionals to deliver their services online without the need for technical skills. (Score: 0.3985)
  -  Hopintech. This tech company is tackling the commute. Hop In provides logistics software services to offer customized corporate shuttle solutions for the daily commute. (Score: 0.3626)
  -  AIRenty. AIRenty is a SaaS PropTech platform which streamline the Real Estate rental process with the help of AI. By automating and simplifying the workflow we make the search activity more enjoyable and less time consuming for everyone! (Score: 0.

## Perform search using the FAISS library

In [7]:
#Finding the closest sentences using the FAISS library 
index = faiss.IndexFlatL2(corpus_embeddings.shape[1]) # build the index 
index.add(np.array(corpus_embeddings).astype('float32')) # add embeddings to the index

In [9]:
# Actual searching using the index
top_k = 5
for query in queries:
    query_embedding = model.encode(query)
    D, I = index.search(np.array([query_embedding]).astype('float32'), top_k)
    # Return the results in order
    print("\n********************")
    print("Query:", query)
    print("\nTop 5 most similar entries in corpus:")
    for i in range(top_k):
        print("  - ",corpus[I[0][i]])


********************
Query: online scheduling and manpower management

Top 5 most similar entries in corpus:
  -  MeetFox. MeetFox offers individual service professionals a simple, one-stop solution to digitize their business. With the vision of becoming the Shopify for professional services, MeetFox combines online scheduling, seamless video calls, and instant payments in one product that can be plugged in on any website and allowing currently 9000 professionals to deliver their services online without the need for technical skills.
  -  AIRenty. AIRenty is a SaaS PropTech platform which streamline the Real Estate rental process with the help of AI. By automating and simplifying the workflow we make the search activity more enjoyable and less time consuming for everyone!
  -  Getchat.app. Engagement with your website visitor and turning them into customers is key. This startup helps you to chat with your visitors on your site using WhatsApp. Getchat.app got the product of the day wit